In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Read data as dataframe
df = pd.read_csv("/home/hildurk/data/3-Simon/promoterDHS_distalDHS_pairs.csv")

In [ ]:
from Bio import SeqIO

# Define a function to convert a chromosome string to a genomic sequence
def chrom2seq(chrom):
    """Convert chromosome string "chrom" to a genomic sequence"""

    return list(SeqIO.parse(
        "/home/hildurk/data/3-Simon/hg38.analysisSet.chroms/%s.fa" % chrom, "fasta"))[0].seq.upper()

In [ ]:
chrom2seq('chr1')

In [ ]:
# list of chromosomes
CHROMS = ["chr1", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", 
          "chr18", "chr19", "chr2", "chr20", "chr21", "chr22", "chr3", "chr4", "chr5", "chr6", 
          "chr7", "chr8", "chr9", "chrM", "chrX", "chrY"]

# Make dictionary
CHROM2SEQ = dict(zip(CHROMS,[chrom2seq(chrom) for chrom in CHROMS]))

In [ ]:
seq = CHROM2SEQ['chr1'][925589:925790]
print(seq)

In [ ]:
# Define a function to create a one-hot encoding for a genomic sequence
base2row = {'A':0,'C':1,'G':2,'T':3}
def seq2onehot(seq):
    """Create a one-hot encoding of a nucleotide sequence"""
    
    A = np.zeros((len(seq),4),dtype=bool)
    for i, base in enumerate(seq):
        if base=='N':
            continue
        A[i,base2row[base]] = True
    return A

In [ ]:
seq1h = seq2onehot(seq)
np.shape(seq1h)
print(seq1h[:5,:])
seq[:5]

In [ ]:
# Define a function to find the reverse compliment of a one-hot encoding
def onehot2rc(onehot):
    """Find the one-hot encoding of the reverse compliment
    of the genomic sequence with one-hot encoding onehot"""
    
    A = np.flipud(onehot) # First reverse
    A = np.fliplr(A) # Then take compliment (i.e. switch A&T and C&G)
    #B = np.zeros(np.shape(A),dtype=bool)
    #B[:,0], B[:,1], B[:,2], B[:,3] = A[:,1], A[:,0], A[:,3], A[:,2]
    return A

In [ ]:
A = seq1h[:5,:]
B = onehot2rc(A)
print(A)
print(B)

In [ ]:
np.shape(df)

In [ ]:
"""Sample the same number of linked==1 class as linked==0 class
idx1 = np.where(df["linked"])[0]
idx0 = np.where(df["linked"]==0)[0]
np.random.shuffle(idx1)
np.random.shuffle(idx0)

nsamples_per_class = 20000
print(np.shape(idx1))
print(np.shape(idx0))
idx1 = idx1[:nsamples_per_class]
idx0 = idx0[:nsamples_per_class]
print(np.shape(idx1))
print(np.shape(idx0))

# Save these indices
np.savez("/home/hildurk/BCdata_project3/SampleIdx.npz",
         idx1=idx1, idx0=idx0)
"""
# Load idx1 and idx0 if saved
sample_idx = np.load("/home/hildurk/BCdata_project3/SampleIdx.npz")
idx1, idx0 = sample_idx['idx1'], sample_idx['idx0']

In [ ]:
np.concatenate([idx1, idx0])

In [ ]:
# Take a sample
idx10 = np.concatenate([idx1,idx0])
np.random.shuffle(idx10)
newdf = df.iloc[idx10]
del df
df = newdf
del newdf

#df = df.sample(n=10000) # Comment this out when loading indices!

In [ ]:
df.head()

In [ ]:
# Create new columns for promoter length and distal length
df["promoter_len"] = df["promoterDHSend"]-df["promoterDHSstart"]
df["distal_len"] = df["distalDHSend"]-df["distalDHSstart"]

# Plot histogram of sequence lengths
plt.figure
plt.hist(df["promoter_len"],bins=30,color="r",alpha=0.4,label="promoter")
plt.hist(df["distal_len"],bins=30,color="b",alpha=0.4,label="distal")
plt.legend()
plt.xlabel("base pair")
plt.ylabel("count")
plt.show()

In [ ]:
# Set a maximum sequence length and filter out longer sequences
seqlength = 2000
df = df[(df["promoter_len"]<seqlength) & (df["distal_len"]<seqlength)]

In [ ]:
# Initialize one-hot encoding arrays (X)
N = df.shape[0]
encoding_promoter = np.zeros((N,seqlength,2,4),dtype=bool)
encoding_distal = np.zeros((N,seqlength,2,4),dtype=bool)

# Initialize Y
Y = np.zeros((N,1))

In [ ]:
# Define X (one-hot encoding) and Y (linked) for Keras
row_no = 0
for i,row in df.iterrows():
    seq_promoter = CHROM2SEQ[row.chr][row.promoterDHSstart:row.promoterDHSend]
    seq_distal = CHROM2SEQ[row.chr][row.distalDHSstart:row.distalDHSend]
    try:
        onehot_promoter = seq2onehot(seq_promoter)
        onehot_distal = seq2onehot(seq_distal)
        encoding_promoter[row_no,:row.promoter_len,0,:] = onehot_promoter
        encoding_distal[row_no,:row.distal_len,0,:] = onehot_distal
        encoding_promoter[row_no,:row.promoter_len,1,:] = onehot2rc(onehot_promoter)
        encoding_distal[row_no,:row.distal_len,1,:] = onehot2rc(onehot_distal)
        Y[row_no] = row.linked
        row_no += 1
    except KeyError:
        print(seq_promoter)
        print(seq_distal)
        break
        
del onehot_promoter
del onehot_distal

In [ ]:
print(np.shape(encoding_promoter))
print(np.shape(encoding_distal))
print(np.shape(Y))

In [ ]:
from sklearn.model_selection import train_test_split

"""Test whether we can predict promoter and distal sequences using model

X = np.zeros((2*N,seqlength,2,4),dtype=bool)
Y = np.zeros((2*N,1),dtype=bool)
X[:N,:,:,:] = encoding_promoter
X[N:,:,:,:] = encoding_distal
Y[:N] = np.ones((N,1),dtype=bool)

# Create training, development and test data sets

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train, y_train, test_size=0.33)

#"""

"""
"""
# Create training, development and test data sets

train_idx, test_idx, y_train, y_test = train_test_split(
    np.arange(N), Y, test_size=0.25)

train_idx, dev_idx, y_train, y_dev = train_test_split(
    train_idx, y_train, test_size=0.33)

# Save these so that we don't keep getting new training and test sets

np.savez("/home/hildurk/BCdata_project3/TrainDevTest.npz",
         train_idx=train_idx, dev_idx=dev_idx, test_idx=test_idx,
        y_train=y_train, y_dev=y_dev, y_test=y_test)

"""
# Load saved train, dev, test indices and y vectors
TrainDevTest = np.load("/home/joshscurll/bcdata_Altius_Project/BCdata_project3/TrainDevTest.npz")
train_idx, dev_idx, test_idx = TrainDevTest['train_idx'], TrainDevTest['dev_idx'], TrainDevTest['test_idx']
y_train, y_dev, y_test = TrainDevTest['y_train'], TrainDevTest['y_dev'], TrainDevTest['y_test']
"""

In [ ]:

xp_train = encoding_promoter[train_idx,:,:]
xp_dev = encoding_promoter[dev_idx,:,:]
xp_test = encoding_promoter[test_idx,:,:]

xd_train = encoding_distal[train_idx,:,:]
xd_dev = encoding_distal[dev_idx,:,:]
xd_test = encoding_distal[test_idx,:,:]


In [ ]:
import keras 
from keras import backend as K

import numpy as np

from keras.layers import Dense, Dropout, Input
from keras.layers import Conv1D, Conv2D, GlobalMaxPooling1D, GlobalMaxPooling2D, MaxPooling2D
from keras.models import Model
from keras.constraints import non_neg
from keras import regularizers as regs

from keras.utils import to_categorical

from keras.callbacks import EarlyStopping

In [ ]:
# Define some functions for the model

def reluConv2D(x, filters, kernel_size, name):
    return Conv2D(filters=filters, kernel_size=(kernel_size,1),
                 activation='relu', padding='same', name=name,
                  kernel_constraint=non_neg(),
                 kernel_regularizer=regs.l1(0.001))(x)

def gmp2d(x):
    return GlobalMaxPooling2D()(x)

"""
def columnwise_mp(x):
    return MaxPooling2D(pool_size=(int(np.shape(x)[1]),1))(x)
    #return MaxPooling2D(pool_size=(2000,1))(x)

def reluConv1D(x, filters, kernel_size, name):
    return Conv1D(filters=filters, kernel_size=kernel_size, 
                  activation='relu', padding='same', name=name)(x)

def gmp1d(x):
    return GlobalMaxPooling1D()(x)
"""

def convMP(x, filters, kernel_size, name):
    return gmp2d(reluConv2D(x, filters, kernel_size, name))

In [ ]:
np.shape(encoding_promoter)

In [ ]:
"""
import tensorflow as tf
X = tf.stack(encoding_promoter[:10,:,:,:].astype(float))
Xconv = reluConv2D(X,3,4,'TestConv')
"""

In [ ]:
"""
Xconvgmp = gmp2d(Xconv)
np.shape(Xconvgmp)
"""

In [ ]:
"""
Xconvmp = columnwise_mp(Xconv)
np.shape(Xconvmp)
"""

In [ ]:
"""# Specify the model
onehot = Input(shape=(seqlength,2,4),name='promoterDHSoh')
fingerprint = convMP(onehot,24,6,'ConvPromoterDHS')
prob_association = Dense(1,activation='sigmoid',name='logistic_regr')(fingerprint)"""

In [ ]:
one_hot_encoding_promoter = Input(shape=(seqlength,2,4),name='promDHSoh') #oh is one hot
one_hot_encoding_distal = Input(shape=(seqlength,2,4),name='distalDHSoh') #oh is one hot

In [ ]:
nb_filters = 20 #temp, we'll change it later
filter_len = 6 #temp, we'll change it later

#conv_layer_distal = Conv1D(nb_filters,filter_len,padding='same',name='distalDHSconv',activation='relu')(one_hot_encoding_distal)
fingerprint_distal = convMP(one_hot_encoding_distal,nb_filters,filter_len,'distalDHSconv')#GlobalMaxPooling1D()(conv_layer_distal)

#conv_layer_promoter = Conv1D(nb_filters,filter_len,padding='same',name='promDHSconv',activation='relu')(one_hot_encoding_promoter)
fingerprint_promoter = convMP(one_hot_encoding_promoter,nb_filters,filter_len,'promDHSconv')#GlobalMaxPooling1D()(conv_layer_promoter)

merged_vector = keras.layers.concatenate([fingerprint_distal,fingerprint_promoter],axis=-1)
prob_association = Dense(1,activation='sigmoid',name='logistic_regression')(merged_vector) #one neuron logistic regression

In [ ]:
model = Model(inputs=[one_hot_encoding_distal,one_hot_encoding_promoter],outputs=prob_association)

In [ ]:
#model = Model(inputs=onehot,outputs=prob_association)
model.summary()

In [ ]:
# Get the percentage of nonzeros in Y
#pc = len(np.where(Y)[0]) / len(Y)
#print(pc)

In [ ]:
from keras.optimizers import Adam
from keras import metrics
from keras.callbacks import EarlyStopping,ModelCheckpoint

# Compile the model
model.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['mse','accuracy'])

checkpointer = ModelCheckpoint(filepath='PD_RC_20nf_lambda0001.hdf5', verbose=1, save_best_only=True)

In [ ]:
"""
Fit the model using a dictionary to weight class 1 more highly than class 0
so that both classes are represented equally in the model fitting
"""
import sklearn
#class_weight = {0: 1, 1: min([1./pc-1.,50])}
#class_weight_vec = sklearn.utils.class_weight.compute_class_weight('balanced',[0.,1.],Y)
model.fit([xd_train,xp_train],y_train,batch_size=256,epochs=50,
          validation_split=0.2,
          callbacks=[EarlyStopping(patience=3),checkpointer])

In [ ]:
model.save('PD_RC_20nf_lambda0001.h5')

In [ ]:
from keras.models import load_model
model=load_model('PD_RC_20nf_lambda0001.hdf5')

In [ ]:
"""
Visualizing the filters
"""
plt.rcParams.update({'font.size': 18})
filters=model.get_layer('promDHSconv').get_weights()[0]
fig, axs = plt.subplots(int(np.ceil(np.shape(filters)[3]/3)),3, figsize=(20, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(np.shape(filters)[3]):
    im=axs[i].imshow(filters[:,0,:,i].T)
    plt.colorbar(im,ax=axs[i])
#plt.title('Filters')
plt.show()

In [ ]:
"""
Visualizing the filters
"""
plt.rcParams.update({'font.size': 18})
filters=model.get_layer('distalDHSconv').get_weights()[0]
fig, axs = plt.subplots(int(np.ceil(np.shape(filters)[3]/3)),3, figsize=(20, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(np.shape(filters)[3]):
    im=axs[i].imshow(filters[:,0,:,i].T)
    plt.colorbar(im,ax=axs[i])
#plt.title('Filters')
plt.show()

In [ ]:
np.shape(filters)

In [ ]:
"""
Get the beta values
"""
beta=model.get_layer('logistic_regression').get_weights()[0]
print(beta[15])

In [ ]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [ ]:
"""
ROC score
"""
print('Predicting on development data...')
y_score = model.predict([xd_test,xp_test])
score=roc_auc_score(y_test, y_score, average='macro', sample_weight=None)
print(score)

In [ ]:
from sklearn.metrics import roc_curve, auc

"""
ROC score
"""
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
plt.show()
print('AUC: %f' % roc_auc)

In [ ]:
print(seq_promoter)

In [ ]:
# summarize history for accuracy
plt.plot(model.history.history['acc'])
plt.plot(model.history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()